In [1]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

from constant_prompts import make_constant_prompts
from util.generate import generate_fast # adding

import json
import pandas as pd
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [2]:
MODEL_NAME = "EleutherAI/gpt-j-6B" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [4]:
with open('data/counterfact-selected-qual.json') as json_file:
   cf_data = json.load(json_file)

reldf = pd.read_csv("counterfact/counterfact-selected-relations.csv")
print(len(cf_data))

172


In [5]:
from evaluate import load
bertscore = load("bertscore")

# def generate_text(texts, model, tok):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens=15,
#                                        num_return_sequences = 5,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)


def gen_constant_prompts(subject, subject_type, orig_target, model, tokenizer, top_n = 10):

    prompts = make_constant_prompts(subject, subject_type)
    
    generations = generate_fast(model, tokenizer, prompts, n_gen_per_prompt = 5, max_out_len = 25)
    gens_per = len(generations) // len(prompts)
    out = []

    for i in range(len(prompts)):
        gens = generations[i*gens_per:i*gens_per+gens_per]
        gens = [g[len(prompts[i]):] for g in gens]
        predictions = gens
        references = [orig_target]*len(gens)
        results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased") # "distilbert-base-uncased"
        val = max(results["recall"])

        resdict = dict()
        resdict["prompt"] = prompts[i]
        resdict["gens"] = gens
        resdict["val"] = val

        out.append(resdict)

    out_sorted = sorted(out, key=lambda d: d['val'])

    return(out_sorted[:top_n])

# gen_constant_prompts("Lebron James", "person", "basketball", model, tokenizer, 6)

In [7]:
tokenizer.pad_token = tokenizer.eos_token

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(25563)

for x in tqdm(cf_data):

    rel_id = x["requested_rewrite"]["relation_id"]
    subject = x["requested_rewrite"]["subject"]
    orig_target = x["requested_rewrite"]["target_true"]["str"]
    subject_type = reldf.loc[lambda x: x.relation_id == rel_id].subj_type.item()


    cprompts = gen_constant_prompts(subject, subject_type, orig_target, model, tokenizer, 6)
    x["subj_const_prompts"] = {MODEL_NAME: cprompts}



100%|██████████| 172/172 [04:56<00:00,  1.73s/it]


In [8]:
with open("data/counterfact-selected-qual.json", "w") as file:
    json.dump(cf_data, file, indent=4)

In [27]:
## test stuff

import sentence_transformers as st

def sentence_similarity_matrix(sentences1, sentences2):
    from sentence_transformers import SentenceTransformer, util
    smodel = SentenceTransformer('all-MiniLM-L6-v2')

    #Compute embedding for both lists
    embeddings1 = smodel.encode(sentences1, convert_to_tensor=True)
    embeddings2 = smodel.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    return(cosine_scores)


def avg_sentence_similarity(sentences1, sentences2):
    return(torch.mean(sentence_similarity_matrix(sentences1, sentences2)))


# def generate_sc_text(texts, model, tok, max_new_tokens=15, num_return_sequences = 5):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens = max_new_tokens,
#                                        num_return_sequences = num_return_sequences,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)


def calc_subj_gen_similarity(model, tok, gen_prompts, orig_gens):
    sims = []
    for i in range(len(gen_prompts)):
        gens = generate_fast(model, tok, [gen_prompts[i]], n_gen_per_prompt = 5, max_out_len = 25)
        gens = [g[len(gen_prompts[i]):] for g in gens] # just use the generated part, not the original prompt
        sentence_similarity = avg_sentence_similarity(gens, orig_gens[i])
        sims.append(sentence_similarity.item())

    mean_sim = sum(sims)/len(sims)
    
    return(gens, mean_sim)

In [28]:
tokenizer.pad_token = tokenizer.eos_token
record = cf_data[0]
prompts = [record["subj_const_prompts"][MODEL_NAME][i]["prompt"] for i in range(0,6)]
orig_gens = [record["subj_const_prompts"][MODEL_NAME][i]["gens"] for i in range(0,6)]


# generate_fast(model, tokenizer, [prompts[0]])
calc_subj_gen_similarity(model, tokenizer, prompts, orig_gens)

(['    $1,000,000 worth of cocaine from the       ',
  ' Category:Living people\nCategory:1954 births\nCategory:British male sailors (sport',
  '\nhis first guitar when he was 14 and played with the school band. He went on to play in',
  '\nhis first house for $25,000 in the late 1970s. He lived in it for three',
  ' \na new BMW M3 George Michael bought a new BMW M3 The M'],
 0.31336965163548786)

In [21]:
orig_gens

[[" St. George's School, Liverpool, and the University of Sheffield, where",
  " King's School, Bruton, Somerset, and at the Royal College of",
  " the independent St George's School, Weybridge, Surrey. He then",
  " St. John's School, Leatherhead and the Royal College of Art,",
  ' the Cambridge High School for Boys in New Addington, London. At Cambridge'],
 [' a high-school teacher, his father a high-school principal, and',
  ' the singer Grace Jones; his father was Afro-Guyanese.',
  ' a devout Christian and taught her son her beliefs. She raised her son in',
  " a successful interior decorator. George Michael's father was a successful architect.",
  ' born in London, England. He was born on May 27, 1958.'],
 [' a music composer and his mother, Cynthia, was a successful model and beauty',
  ' a well-known and much-respected jazz musician, composer and arranger',
  ' a doctor.\n\nGeorge Michael was born in north London on September 21',
  ' the musician George Michael.\n\nI remember 

'EleutherAI/gpt-j-6B'